In [1]:
torch.setdefaulttensortype('torch.FloatTensor')
torch.manualSeed(666)

In [2]:
mnist = require 'mnist'

train = mnist.traindataset()
test = mnist.testdataset()

local mean, std

for _,set in ipairs{train, test} do
    set.data = set.data:float()
    
    if not mean then
        mean = train.data:mean()
        std = train.data:std()
    end
    set.data:add(-mean)
    set.data:div(std)
    
    set.label:add(1)
end

collectgarbage()

In [10]:
-- ****** Convolutional net ******

require 'nn'

net = nn.Sequential()

net:add(nn.SpatialConvolution(1, 64, 5, 5, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())

net:add(nn.SpatialMaxPooling(2, 2, 2, 2))

net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))
net:add(nn.ReLU())
net:add(nn.SpatialConvolution(64, 64, 3, 3, 1, 1, 0, 0))
net:add(nn.SpatialBatchNormalization(64))

net:add(nn.View(64, -1):setNumInputDims(3))
net:add(nn.Mean(3, 3))
net:add(nn.BatchNormalization(64))

net:add(nn.Linear(64, 10))

params, gradParams = net:getParameters()

collectgarbage()

In [5]:
-- ****** Linear classifier ******

require 'nn'

net = nn.Sequential()

net:add(nn.View(-1):setNumInputDims(3))
net:add(nn.Linear(28*28, 10))

params, gradParams = net:getParameters()

collectgarbage()

In [16]:
-- ****** Haar features ******

require 'nn'

IntegralSmartNorm = nil
debug.getregistry()['IntegralSmartNorm'] = nil 
package.loaded['IntegralSmartNorm'] = nil
require 'IntegralSmartNorm'

net = nn.Sequential()

net:add(IntegralSmartNorm(1, 64, 28, 28))
net:add(nn.SpatialConvolution(64, 64, 1,1, 4,4))
net:add(nn.View(64*7*7):setNumInputDims(3))
-- net:add(nn.Mean(3, 3))
net:add(nn.Linear(64*7*7, 10))

int = net:get(1)
int.normalize = false
int.exact = false
int.updateGradInput = function() end --function(self, input) self.gradInput:resizeAs(input) end

accBatch = true

params, gradParams = net:getParameters()

collectgarbage()

In [4]:
require 'optim'

datasetIdx = 1

optimState = {
    learningRate = 1e-2,
    momentum = 0.9,
    nesterov = true,
    dampening = 0
}

crit = nn.CrossEntropyCriterion()

In [5]:
logger = optim.Logger('MNIST/mnist-acc-integral.log')
logger:setNames{'Training loss'}
logger:style{'-'}
-- logger:showPlot(false)

In [6]:
function fillBatch(batch, labels, set, currIdx, idxs)
    for k = 1,batch:size(1) do
        batch[k]:copy(set.data[idxs and idxs[currIdx] or currIdx])
        labels[k] = set.label[idxs and idxs[currIdx] or currIdx]
        
        currIdx = currIdx + 1
        if currIdx > set.size then currIdx = 1 end
    end
    
    return currIdx
end

In [17]:
batchSize = 128
batch = torch.FloatTensor(batchSize, 1, 28, 28)
labels = torch.IntTensor(batchSize)

function randomShuffle(t)
  for i = 1,#t do
    local j = math.random(i, #t)
    t[i], t[j] = t[j], t[i]
  end
end

idx = {}
for i = 1,train.data:size(1) do
    idx[i] = i
end

for iter = 1,5 do
    local oldIdx = datasetIdx
    datasetIdx = fillBatch(batch, labels, train, datasetIdx, idx)
    if datasetIdx < oldIdx then
        print('New epoch')
        randomShuffle(idx)
        datasetIdx = 1
    end
    
    local loss
    
    timer = torch.Timer()
    if accBatch then
        net:zeroGradParameters()
        loss = 0
        for k = 1,batchSize do
            loss = loss + crit:forward(net:forward(batch[k]), labels[k])
            crit:backward(net.output, labels[k])
            net:backward(batch[k], crit.gradInput, 1 / batchSize)
        end
        loss = loss / batchSize
    else
        loss = crit:forward(net:forward(batch), labels)
        crit:backward(net.output, labels)
        net:zeroGradParameters()
        net:backward(batch, crit.gradInput)
    end
    print(timer:time().real .. ' sec')
    
    local feval = function(x)
        return batchLoss, gradParams
    end

    optim.sgd(feval, params, optimState)
    
    logger:add{loss}
    if iter % 1 == 0 or iter == 469 then
        logger:plot()
    end
    
    collectgarbage() 
end

4.3244831562042 sec	


3.6126310825348 sec	


4.262176990509 sec	


4.1294159889221 sec	


2.984934091568 sec	


In [7]:
-- Test
require 'xlua'

local testBatchSize = 100
assert(test.size % testBatchSize == 0)

local correctCount = 0
local batchCount = 0
local testDatasetIdx = 1
local batch = torch.FloatTensor(testBatchSize, 1, 28, 28)
local labels = torch.LongTensor(testBatchSize)

-- timer = torch.Timer()
repeat
    batchCount = batchCount + 1
    xlua.progress(testBatchSize * batchCount, test.size)
    
    testDatasetIdx = fillBatch(batch, labels, test, testDatasetIdx)
    if accBatch then
        for k = 1,testBatchSize do
            net:forward(batch[k])
            local answer = select(2, net.output:max(1))[1]
            correctCount = correctCount + (answer == labels[k] and 1 or 0)
        end
    else
        net:forward(batch)
        local answer = select(2, net.output:max(2))
        correctCount = correctCount + answer:eq(labels):sum()
    end
    
until testDatasetIdx < testBatchSize
-- print(timer:time().real .. ' seconds')

print(correctCount / (batchCount * testBatchSize) * 100 .. '%')

 [..............................................]                                ETA: 0ms | Step: 0ms 

0.89568591117859 seconds	
6%	


One 100-batch forward pass time:
* convnet: 1.357259035110
* Haar features (normalize = true): 11.31303405761
* normalize = false: 9.618772983551
* no cv.integral: 9.422287940979
* no forwardCFunction: 0.73264694213867
* no OpenMP: 0.89568591117859

In [25]:
net:clearState()
torch.save('mnist-linear.t7', net)
torch.save('mnist-linear-optimstate.t7', optimState)